## 3.4　Pandas数值运算方法

NumPy 的基本能力之一是快速对每个元素进行运算，既包括基本算术运算（加、减、乘、除），也包括更复杂的运算（三角函数、指数函数和对数函数等）。
<br>Pandas 继承了NumPy的功能，在2.3 节介绍过的通用函数是关键。

但是Pandas 也实现了一些高效技巧：
* 对于一元运算（像函数与三角函数），这些通用函数将在输出结果中保留索引和列标签；
* 而对于二元运算（如加法和乘法），Pandas 在传递通用函数时会自动对齐索引进行计算。

这就意味着，保存数据内容与组合不同来源的数据——两处在NumPy 数组中都容易出错的地方——变成了Pandas 的杀手锏。
<br>后面还会介绍一些关于一维Series 和二维DataFrame 的便捷运算方法。

### 3.4.1　通用函数：保留索引

因为Pandas 是建立在NumPy 基础之上的，所以NumPy 的通用函数同样适用于Pandas 的Series 和DataFrame 对象

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


如果对这两个对象的其中一个使用NumPy 通用函数，生成的结果是另一个保留索引的Pandas 对象：

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


### 3.4.2　通用函数：索引对齐

当在两个Series 或DataFrame 对象上进行二元计算时，Pandas 会在计算过程中对齐两个对
象的索引。当你处理不完整的数据时，这一点非常方便。

#### 3.4.2.1 Series索引对齐

假如你要整合两个数据源的数据，其中一个是美国面积最大的三个州的面积数据，另一个是美国人口最多的三个州的人口数据：

In [6]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
'New York': 19651127}, name='population')

如果用人口除以面积会得到什么样的结果：

In [7]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

结果数组的索引是两个输入数组索引的并集。

In [8]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

**对于缺失位置的数据，Pandas 会用NaN 填充，表示“此处无数”。**

这种索引对齐方式是通过Python 内置的集合运算规则实现的，任何缺失值默认都用NaN 填充：

In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

如果用NaN值不是我们想要的结果，那么可以用适当的对象方法代替运算符。

In [10]:
# 例如，A.add(B) 等价于A + B，也可以设置参数自定义A 或B 缺失的数据：
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

#### 3.4.2.2 DataFrame索引对齐

在计算两个DataFrame 时，类似的索引对齐规则也同样会出现在共同（并集）列中：

In [11]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [12]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [13]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


你会发现，两个对象的行列索引可以是不同顺序的，**结果的索引会自动按顺序排列**。<br>在Series 中，我们可以通过运算符方法的fill_value 参数自定义缺失值。

这里，我们将用A中所有值的均值来填充缺失值（计算A 的均值需要用stack 将二维数组压缩成一维数组）：

In [16]:
fill = A.stack().mean()
print("A.mean is: ", fill)
A.add(B, fill_value=fill)

A.mean is:  4.5


,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


表3-1：Python运算符与Pandas方法的映射关系

| Python运算符 | Pandas方法|
| :- | -: |
| + | add() |
| - | sub()、subtract() |
| * | mul()、multiply() |
| / | truediv()、div()、divide() |
| // | floordiv() |
| % | mod() |
| ** | pow() |

### 3.4.3　通用函数：DataFrame与Series的运算

DataFrame 和Series 的运算规则，与NumPy 中二维数组与一维数组的运算规则是一样的。

来看一个常见运算，让一个二维数组减去自身的一行数据：

In [18]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [19]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

根据NumPy 的广播规则（详情请参见2.5 节），让二维数组减自身的一行数据会按行计算。

在Pandas 里默认也是按行运算的：

In [21]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


如果你想按列计算，那么就需要利用前面介绍过的运算符方法，通过axis 参数设置：

In [22]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


DataFrame / Series 的运算与前面介绍的运算一样，结果的索引都会自动对齐：

In [25]:
df

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


In [23]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int32

In [24]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


这些行列索引的保留与对齐方法说明Pandas 在运算时会一直保存这些数据内容，从而避免在处理数据类型有差异和/ 或维度不一致的NumPy 数组时可能遇到的问题